## Data Exploration / Cleaning

In [1]:
import pandas as pd

# Load the datasets

train_df = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\train.csv')
eval_df = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\eval.csv')
metros = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\usmetros.csv')

pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_rows', None) # Show all rows

In [2]:
print(train_df.shape)
print(eval_df.shape)
print(metros.shape)

(585244, 39)
(149424, 39)
(387, 11)


In [3]:
train_df['city_full'].value_counts().head(5)

city_full
New York-Newark-Jersey City       78020
Chicago-Naperville-Elgin          35344
Los Angeles-Long Beach-Anaheim    33840
Philadelphia-Camden-Wilmington    31396
DC_Metro                          29516
Name: count, dtype: int64

### Map cities to Lat/Long
- The goal is to use Lattitude and longitude instead of cities for our ML models

In [4]:
# ============================
# 2. Fix city name mismatches
# ============================
city_mapping = {
    'Las Vegas-Henderson-Paradise': 'Las Vegas-Henderson-North Las Vegas',
    'Denver-Aurora-Lakewood': 'Denver-Aurora-Centennial',
    'Houston-The Woodlands-Sugar Land': 'Houston-Pasadena-The Woodlands',
    'Austin-Round Rock-Georgetown': 'Austin-Round Rock-San Marcos',
    'Miami-Fort Lauderdale-Pompano Beach': 'Miami-Fort Lauderdale-West Palm Beach',
    'San Francisco-Oakland-Berkeley': 'San Francisco-Oakland-Fremont',
    'DC_Metro': 'Washington-Arlington-Alexandria',
    'Atlanta-Sandy Springs-Alpharetta': 'Atlanta-Sandy Springs-Roswell'
}

metros["join_key"] = metros["metro_full"].str.split(',').str[0].str.strip()

In [5]:
def clean_and_merge(df: pd.DataFrame, metros_prepared: pd.DataFrame) -> pd.DataFrame:
    """Apply city name fixes, merge lat/lng from prepared metros, drop helper cols."""
    
    # Apply city name fixes
    df["city_full"] = df["city_full"].replace(city_mapping)
    
    # Merge using the join_key we created outside the function
    df = df.merge(
        metros_prepared[["join_key", "metro_full", "lat", "lng"]],
        how="left",
        left_on="city_full",
        right_on="join_key"
    )

    # Clean up: Drop the 'join_key' and the original 'metro_full' column 
    # to keep only your original 'city_full' + the new lat/lng
    df.drop(columns=["join_key", "metro_full"], inplace=True)

    # Log any cities that still didn’t match
    missing = df[df["lat"].isnull()]["city_full"].unique()
    if len(missing) > 0:
        print("⚠️ Still missing lat/lng for:", missing)
    else:
        print("✅ All cities matched with metros dataset.")

    return df

In [6]:
# apply cleaning and merging to train and eval datasets
train_df = clean_and_merge(train_df, metros)
eval_df = clean_and_merge(eval_df, metros)

✅ All cities matched with metros dataset.
✅ All cities matched with metros dataset.
